<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_groundCover_intercepts_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [2]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

## Database Connection

In [3]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [4]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [5]:
billing <- bq_test_project()

### vegetation_point_intercept_gridVeg

In [6]:
con_point_intercept <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [7]:
dbListTables(con_point_intercept)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_point_intercept_ground"    
[5] "gridVeg_point_intercept_vegetation" "gridVeg_shrub_tree"                
[7] "gridVeg_survey_metadata"

In [8]:
ground_point_intercept_sql <- 
"
  SELECT
    survey_ID,
    grid_point,
    intercept_ground_code,
    COUNT(intercept_ground_code) / 2 AS intercepts_pct
  FROM 
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_point_intercept_ground`
  GROUP BY
    survey_ID, grid_point, intercept_ground_code
"

In [9]:
bq_ground_point_intercept <- bq_project_query(billing, ground_point_intercept_sql)

In [10]:
tb_ground_point_intercept <- bq_table_download(bq_ground_point_intercept)

In [11]:
df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept)

In [12]:
head(df_ground_point_intercept, n=4)

,survey_ID,grid_point,intercept_ground_code,intercepts_pct
,<chr>,<int>,<chr>,<dbl>
1,2,373,NA,0
2,338,141,NA,0
3,433,332,NA,0
4,569,26,NA,0


In [13]:
df_ground_point_intercept %>%
  filter(survey_ID == '027F5AE0-F90B-4E4D-81D8-E7674D600243')

survey_ID,grid_point,intercept_ground_code,intercepts_pct
<chr>,<int>,<chr>,<dbl>
027F5AE0-F90B-4E4D-81D8-E7674D600243,323,G,3.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,323,L,42.5
027F5AE0-F90B-4E4D-81D8-E7674D600243,323,R,1.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,323,BV,5.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,323,M/L,48.5


### gridVeg_survey_metadata

In [14]:
survey_metadata_sql <-
"
  SELECT
    survey_ID,
    year,
    survey_sequence
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
"

In [15]:
bq_survey_metadata <- bq_project_query(billing, survey_metadata_sql)

In [16]:
tb_survey_metadata <- bq_table_download(bq_survey_metadata)

In [17]:
df_survey_metadata <- as.data.frame(tb_survey_metadata)

In [18]:
head(df_survey_metadata, n=4)

,survey_ID,year,survey_sequence
,<chr>,<int>,<chr>
1,F31C56A8-912D-410C-A17D-4C2DD75F71A4,2016,2016
2,A19E87E6-A89C-4993-B550-802226730D54,2016,2016
3,6F1D71D3-9F87-4C93-B179-A12C8938D18D,2016,2016
4,9C67C9F1-1E89-4FD2-ADC0-0390E0022D62,2016,2016


### ground_cover_metadata

In [19]:
sql_ground_meta <- 
"
SELECT
  intercept_ground_code,
  ground_group
FROM
  `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_ground_cover_metadata`
"

In [20]:
bq_ground_meta <- bq_project_query(billing, sql_ground_meta)

In [21]:
tb_ground_meta <- bq_table_download(bq_ground_meta)

In [22]:
df_ground_meta <- as.data.frame(tb_ground_meta)

In [23]:
df_ground_meta %>% arrange(intercept_ground_code)

intercept_ground_code,ground_group
<chr>,<chr>
BG,inorganic
BV,living vegetation
G,inorganic
L,dead vegetation
LIC,living vegetation
M,living vegetation
M/L,living vegetation
OTHER,inorganic
R,inorganic


# Wrangle

In [24]:
# show unique intercept ground codes
df_ground_point_intercept %>%
  distinct(intercept_ground_code) %>% 
  arrange(intercept_ground_code)

intercept_ground_code
<chr>
BG
BV
G
L
LIC
M
M/L
OTHER
R


## Fill missing ground code

In [27]:
df_ground_point_intercept_fill <- df_ground_point_intercept %>%
  # include all intercept_ground variables for each survey_ID if not previously present set to 0
  complete(survey_ID, nesting(intercept_ground_code), fill= list(intercepts_pct = 0)) %>%
  fill(grid_point) %>%
  filter(intercept_ground_code != 'NA') %>% 
  glimpse()

Rows: 22,392
Columns: 4
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5F…
$ intercept_ground_code <chr> "BG", "BV", "G", "L", "LIC", "M", "M/L", "OTHER…
$ grid_point            <int> 285, 285, 285, 285, 285, 285, 285, 285, 285, 28…
$ intercepts_pct        <dbl> 15.5, 8.5, 8.5, 62.0, 0.0, 1.5, 0.0, 0.0, 1.5, …


In [29]:
df_ground_point_intercept_fill %>%
  filter(survey_ID == '027F5AE0-F90B-4E4D-81D8-E7674D600243')

survey_ID,intercept_ground_code,grid_point,intercepts_pct
<chr>,<chr>,<int>,<dbl>
027F5AE0-F90B-4E4D-81D8-E7674D600243,BG,23,0.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,BV,323,5.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,G,323,3.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,L,323,42.5
027F5AE0-F90B-4E4D-81D8-E7674D600243,LIC,323,0.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,M,323,0.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,M/L,323,48.5
027F5AE0-F90B-4E4D-81D8-E7674D600243,OTHER,323,0.0
027F5AE0-F90B-4E4D-81D8-E7674D600243,R,323,1.0


## Join datasets

In [ ]:
df_gridVeg_groundCover_intercepts <- 
  df_ground_point_intercept_fill %>%
  left_join(df_survey_metadata, by = "survey_ID") %>% 
  left_join(df_ground_meta, by = "intercept_ground_code") %>% 
  select(survey_ID, year, survey_sequence, grid_point, intercept_ground_code, ground_group, intercepts_pct) %>% 
  glimpse()

Rows: 22,392
Columns: 7
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5F…
$ year                  <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence       <chr> "2016", "2016", "2016", "2016", "2016", "2016",…
$ grid_point            <int> 285, 285, 285, 285, 285, 285, 285, 285, 285, 28…
$ intercept_ground_code <chr> "BG", "BV", "G", "L", "LIC", "M", "M/L", "OTHER…
$ ground_group          <chr> "inorganic", "living vegetation", "inorganic", …
$ intercepts_pct        <dbl> 15.5, 8.5, 8.5, 62.0, 0.0, 1.5, 0.0, 0.0, 1.5, …


## QC
* No rows contain missing data, and this makes sense since `complete()` was used to fill response variables.
* Numeric variables appear correct in the summary

In [ ]:
df_gridVeg_groundCover_intercepts[which(!complete.cases(df_gridVeg_groundCover_intercepts)), ]

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>


In [ ]:
summary(df_gridVeg_groundCover_intercepts)

  survey_ID              year      survey_sequence      grid_point   
 Length:22392       Min.   :2011   Length:22392       Min.   :  1.0  
 Class :character   1st Qu.:2011   Class :character   1st Qu.:136.0  
 Mode  :character   Median :2015   Mode  :character   Median :288.0  
                    Mean   :2014                      Mean   :285.7  
                    3rd Qu.:2016                      3rd Qu.:430.0  
                    Max.   :2017                      Max.   :583.0  
 intercept_ground_code ground_group       intercepts_pct  
 Length:22392          Length:22392       Min.   : 0.000  
 Class :character      Class :character   1st Qu.: 0.000  
 Mode  :character      Mode  :character   Median : 0.000  
                                          Mean   : 5.545  
                                          3rd Qu.: 2.000  
                                          Max.   :99.500  

In [ ]:
df_gridVeg_groundCover_intercepts %>% filter(survey_ID == 234)

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>
234,2011,2011-12,48,BG,inorganic,6.5
234,2011,2011-12,48,BV,living vegetation,19.5
234,2011,2011-12,48,G,inorganic,16.5
234,2011,2011-12,48,L,dead vegetation,41.5
234,2011,2011-12,48,LIC,living vegetation,0.0
234,2011,2011-12,48,M,living vegetation,0.0
234,2011,2011-12,48,M/L,living vegetation,12.0
234,2011,2011-12,48,OTHER,inorganic,0.0
234,2011,2011-12,48,R,inorganic,3.0


# Output

In [ ]:
write_csv(df_gridVeg_groundCover_intercepts, path = "gridVeg_groundCover_intercepts_WRANGLE.csv")